In [1]:
import sys
import numpy as np
import math
import time
import json
import openravepy

from src.Robots.Models import FetchOpenRaveRobotModel
import Config

sys.path.append(Config.TEST_DIR + 'EnvGenerators/' + Config.DOMAIN + '/')
import model_file_generator as MFG

ENV_GEN_DIR = Config.TEST_DIR + 'EnvGenerators/' + Config.DOMAIN + '/' 
MODELS_DIR = Config.TEST_DIR + 'EnvGenerators/' + Config.DOMAIN + '/Models/'
CURRENT_ENV_DIR = Config.TEST_DIR + 'EnvGenerators/' + Config.DOMAIN + '/current_env/'

env = openravepy.Environment()
env.SetViewer('qtcoin')

openravepy.misc.DrawAxes(env, [1,0,0,0,0,0,0])

model_files = {}

config = 4  # np.random.randint(1, 5)
print "config: {}".format(config)


config: 2


In [2]:
def set_floor_wall_shelf_poses(config):
    '''
    Set the position of the floor, walls and tool shelf in the environment based on the config
    The floor, room and shelf models must have already been loaded in the environment
    The function only performs the required rotations and translations 
    '''
    model_name = 'room1_floor'
    floor1 = env.GetKinBody(model_name)

    floor_aabb = floor1.ComputeAABB()
    floor_x_len = floor_aabb.extents()[0] * 2
    floor_y_len = floor_aabb.extents()[1] * 2
    floor_z_len = floor_aabb.extents()[2] * 2

    model_name = 'room2_floor'
    floor2 = env.GetKinBody(model_name)

    # rotate around z axis
    rot_angle = 0
    floor1_rotation_matrix = np.identity(4)
    floor1_rotation_matrix[0][0] = math.cos(rot_angle)
    floor1_rotation_matrix[0][1] = -(math.sin(rot_angle))
    floor1_rotation_matrix[1][0] = math.sin(rot_angle)
    floor1_rotation_matrix[1][1] = math.cos(rot_angle)

    floor2_rotation_matrix = np.identity(4)
    floor2_rotation_matrix[0][0] = math.cos(rot_angle)
    floor2_rotation_matrix[0][1] = -(math.sin(rot_angle))
    floor2_rotation_matrix[1][0] = math.sin(rot_angle)
    floor2_rotation_matrix[1][1] = math.cos(rot_angle)
    
    # shift the floor down by it's z-height so we don't need to move other models along z-axis
    floor1_rotation_matrix[2][3] = -floor_z_len - 0.01
    floor2_rotation_matrix[2][3] = -floor_z_len - 0.01
    
    # 0.001 offset is maintained so models aren't in collision with each other in openrave
    # Get displacement according to config
    if config == 1:    
        floor2_rotation_matrix[0][3] = -floor_x_len - 0.001
    elif config == 2:
        floor2_rotation_matrix[1][3] = floor_y_len + 0.001
    elif config == 3:
        floor2_rotation_matrix[0][3] = floor_x_len + 0.001
    elif config == 4:
        floor2_rotation_matrix[1][3] = -floor_y_len - 0.001
    
    # Set floor 2 transform
    floor2_trans = floor2.GetTransform()
    rotated_pose = np.matmul(floor2_trans, floor2_rotation_matrix)
    Tz = openravepy.matrixFromPose(rotated_pose)
    floor2.SetTransform(Tz)

    # Set floor 1 transform
    floor1_trans = floor1.GetTransform()
    rotated_pose = np.matmul(floor1_trans, floor1_rotation_matrix)
    Tz = openravepy.matrixFromPose(rotated_pose)
    floor1.SetTransform(Tz)

    # Position walls
    model_name = 'room1_walls'
    wall1 = env.GetKinBody(model_name)

    wall_aabb = wall1.ComputeAABB()
    wall_x_len = wall_aabb.extents()[0] * 2
    wall_y_len = wall_aabb.extents()[1] * 2
    wall_z_len = wall_aabb.extents()[2] * 2
    
    model_name = 'room2_walls'
    wall2 = env.GetKinBody(model_name)
    
    # Decide rotation angle for walls
    if config == 1:
        wall1_rot_angle = 0
        wall2_rot_angle = np.pi
    elif config == 2:
        wall1_rot_angle = 0
        wall2_rot_angle = np.pi
    elif config == 3:
        wall1_rot_angle = np.pi
        wall2_rot_angle = 0
    elif config == 4:
        wall1_rot_angle = np.pi
        wall2_rot_angle = 0
        
    # rotate around z axis
    wall1_rotation_matrix = np.identity(4)
    wall1_rotation_matrix[0][0] = math.cos(wall1_rot_angle)
    wall1_rotation_matrix[0][1] = -(math.sin(wall1_rot_angle))
    wall1_rotation_matrix[1][0] = math.sin(wall1_rot_angle)
    wall1_rotation_matrix[1][1] = math.cos(wall1_rot_angle)
    
    # rotate around z axis
    wall2_rotation_matrix = np.identity(4)
    wall2_rotation_matrix[0][0] = math.cos(wall2_rot_angle)
    wall2_rotation_matrix[0][1] = -(math.sin(wall2_rot_angle))
    wall2_rotation_matrix[1][0] = math.sin(wall2_rot_angle)
    wall2_rotation_matrix[1][1] = math.cos(wall2_rot_angle)
    
    if config == 1:
        wall2_rotation_matrix[1][3] = -wall_y_len
        wall2_rotation_matrix[0][3] = -0.001
    elif config == 2:
        wall2_rotation_matrix[0][3] = wall_x_len
        wall2_rotation_matrix[1][3] = 0.001
    elif config == 3:
        wall1_rotation_matrix[0][3] = wall_x_len - 0.001
        wall1_rotation_matrix[1][3] = -wall_y_len
        wall2_rotation_matrix[0][3] = wall_x_len
    elif config == 4:
        wall1_rotation_matrix[0][3] = wall_x_len
        wall1_rotation_matrix[1][3] = -wall_y_len + 0.001
        wall2_rotation_matrix[1][3] = -wall_y_len
    
    wall1_trans = wall1.GetTransform()
    rotated_pose = np.matmul(wall1_trans, wall1_rotation_matrix)
    Tz = openravepy.matrixFromPose(rotated_pose)
    wall1.SetTransform(Tz)

    wall2_trans = wall2.GetTransform()
    rotated_pose = np.matmul(wall2_trans, wall2_rotation_matrix)
    Tz = openravepy.matrixFromPose(rotated_pose)
    wall2.SetTransform(Tz)

    model_name = 'tool_shelf'
    shelf = env.GetKinBody(model_name)
    shelf_aabb = shelf.ComputeAABB()
    shelf_x_len = shelf_aabb.extents()[0] * 2
    shelf_y_len = shelf_aabb.extents()[1] * 2
    shelf_z_len = shelf_aabb.extents()[2] * 2

    if config == 1 or config == 2 or config == 3:
        rot_angle = -np.pi/2
    elif config == 4:
        rot_angle = np.pi/2

    # rotate around z axis
    shelf_rotation_matrix = np.identity(4)
    shelf_rotation_matrix[0][0] = math.cos(rot_angle)
    shelf_rotation_matrix[0][1] = -(math.sin(rot_angle))
    shelf_rotation_matrix[1][0] = math.sin(rot_angle)
    shelf_rotation_matrix[1][1] = math.cos(rot_angle)

    if config == 1 or config == 2 or config == 3:
        shelf_rotation_matrix[0][3] = floor_x_len/2
        shelf_rotation_matrix[1][3] = -floor_y_len + shelf_x_len*2
    elif config == 4:
        shelf_rotation_matrix[0][3] = floor_x_len/2
        shelf_rotation_matrix[1][3] = -shelf_x_len*2

    shelf_trans = shelf.GetTransform()
    rotated_pose = np.matmul(shelf_trans, shelf_rotation_matrix)
    Tz = openravepy.matrixFromPose(rotated_pose)
    shelf.SetTransform(Tz)

# Load and position floor, walls and tool shelf models

In [3]:
floor_model = MODELS_DIR + 'floor.dae'
walls_door_long_model = MODELS_DIR + 'walls_door_long.dae'
walls_door_short_model = MODELS_DIR + 'walls_door_short.dae'
tool_shelf_model = MODELS_DIR + 'tool_shelf.dae'
walls_model = None

if config == 1 or config == 3:
    walls_model = walls_door_long_model
else:
    walls_model = walls_door_short_model

# Place floor and walls
model_name = 'room1_floor'
dae15, dae14, sdf = MFG.generate_model_files('room1_floor', floor_model, CURRENT_ENV_DIR, MODELS_DIR)
room1_floor = env.ReadKinBodyURI(dae15)
model_files[model_name] = {'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
env.AddKinBody(room1_floor)

model_name = 'room1_walls'
dae15, dae14, sdf = MFG.generate_model_files('room1_walls', walls_model, CURRENT_ENV_DIR, MODELS_DIR)
model_files[model_name] = {'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
room1_walls = env.ReadKinBodyURI(dae15)
env.AddKinBody(room1_walls)

model_name = 'room2_floor'
dae15, dae14, sdf = MFG.generate_model_files('room2_floor', floor_model, CURRENT_ENV_DIR, MODELS_DIR)
model_files[model_name] = {'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
room2_floor = env.ReadKinBodyURI(dae15)
env.AddKinBody(room2_floor)

model_name = 'room2_walls'
dae15, dae14, sdf = MFG.generate_model_files('room2_walls', walls_model, CURRENT_ENV_DIR, MODELS_DIR)
model_files[model_name] = {'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
room2_walls = env.ReadKinBodyURI(dae15)
env.AddKinBody(room2_walls)

model_name = 'tool_shelf'
dae15, dae14, sdf = MFG.generate_model_files('tool_shelf', tool_shelf_model, CURRENT_ENV_DIR, MODELS_DIR)
model_files[model_name] = {'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
tool_shelf = env.ReadKinBodyURI(dae15)
env.AddKinBody(tool_shelf)

set_floor_wall_shelf_poses(config)

In [4]:
def add_machine_to_env(machine_base_model_name, x, y, rot_angle, workstation_model_name, machine_model_name):
    '''
    Positions the given model in the environment. Used to position machines of type 1,2,3

    machine_base_model_name: KinBody name of the base model on top of which the machine will be placed
    x: x coordinate to position the origin of the model at
    y: y coordinate to position the origin of the model at
    rot_angle: rotation of the model
    workstation_model_name: name of the workstation model if any (can be None)
    machine_model_name: name of the machine model
    '''
    global env
    
    body = env.GetKinBody(machine_base_model_name)
    pose = openravepy.poseFromMatrix(body.GetTransform())
    pose[4] = x
    pose[5] = y

    Tz = openravepy.matrixFromPose(pose)
    body.SetTransform(Tz)
    
    body_aabb = body.ComputeAABB()
    body_x_len = body_aabb.extents()[0]*2
    body_y_len = body_aabb.extents()[1]*2
    body_z_len = body_aabb.extents()[2]*2

    # rotate around z axis
    rotation_matrix = np.identity(4)
    rotation_matrix[0][0] = math.cos(rot_angle)
    rotation_matrix[0][1] = -(math.sin(rot_angle))
    rotation_matrix[1][0] = math.sin(rot_angle)
    rotation_matrix[1][1] = math.cos(rot_angle)

    # after rotating need to shift it down since rotation origin is at top left corner
    if rot_angle == np.pi / 2:
        rotation_matrix[1][3] = -body_x_len
    elif rot_angle == np.pi:
        rotation_matrix[1][3] = -body_y_len
        rotation_matrix[0][3] = +body_x_len
    elif rot_angle == -np.pi / 2:
        rotation_matrix[0][3] = +body_y_len


    body_trans = body.GetTransform()
    rotated_pose = np.matmul(body_trans, rotation_matrix)

    Tz = openravepy.matrixFromPose(rotated_pose)
    body.SetTransform(Tz)
    
    if workstation_model_name is not None:
        body = env.GetKinBody(workstation_model_name)
        pose = openravepy.poseFromMatrix(body.GetTransform())
        pose[4] = x
        pose[5] = y
        
        Tz = openravepy.matrixFromPose(pose)
        body.SetTransform(Tz)
        
        # rotate around z axis
        rotation_matrix = np.identity(4)
        rotation_matrix[0][0] = math.cos(rot_angle)
        rotation_matrix[0][1] = -(math.sin(rot_angle))
        rotation_matrix[1][0] = math.sin(rot_angle)
        rotation_matrix[1][1] = math.cos(rot_angle)
        
        if rot_angle == 0:
            rotation_matrix[0][3] = -body_x_len - 0.001
        elif rot_angle == np.pi/2:
            rotation_matrix[1][3] = -2 * body_x_len - 0.001
        elif rot_angle == np.pi:
            rotation_matrix[0][3] = 2 * body_x_len + 0.001
            rotation_matrix[1][3] = -body_y_len
        elif rot_angle == -np.pi/2:
            rotation_matrix[0][3] = body_y_len
            rotation_matrix[1][3] = body_x_len + 0.001
            
        body_trans = body.GetTransform()
        rotated_pose = np.matmul(body_trans, rotation_matrix)

        Tz = openravepy.matrixFromPose(rotated_pose)
        body.SetTransform(Tz)
    
    
    if machine_model_name is not None:
        
        body = env.GetKinBody(machine_model_name)
        pose = openravepy.poseFromMatrix(body.GetTransform())
        pose[4] = x
        pose[5] = y
        pose[6] = body_z_len + 0.005
        
        Tz = openravepy.matrixFromPose(pose)
        body.SetTransform(Tz)
        
        machine_rot_angle = rot_angle - (np.pi / 2)
                
        # rotate around z axis
        rotation_matrix = np.identity(4)
        rotation_matrix[0][0] = math.cos(machine_rot_angle)
        rotation_matrix[0][1] = -(math.sin(machine_rot_angle))
        rotation_matrix[1][0] = math.sin(machine_rot_angle)
        rotation_matrix[1][1] = math.cos(machine_rot_angle)
        
        body_aabb = body.ComputeAABB()
        machine_x_len = body_aabb.extents()[0] * 2
        machine_y_len = body_aabb.extents()[1] * 2
        machine_z_len = body_aabb.extents()[2] * 2
        
        if rot_angle == 0:
            rotation_matrix[0][3] = 0.5 * body_x_len
            rotation_matrix[1][3] = -0.5 * body_y_len
        elif rot_angle == np.pi/2:
            pass
            rotation_matrix[0][3] = 0.5 * body_y_len
            rotation_matrix[1][3] = -0.5 * body_x_len
        elif rot_angle == np.pi:
            pass
            rotation_matrix[0][3] = 0.5 * body_x_len
            rotation_matrix[1][3] = -0.5 * body_y_len
        elif rot_angle == -np.pi/2:
            rotation_matrix[0][3] = 0.5 * body_y_len
            rotation_matrix[1][3] = -0.5 * body_x_len
        
        body_trans = body.GetTransform()
        rotated_pose = np.matmul(body_trans, rotation_matrix)

        Tz = openravepy.matrixFromPose(rotated_pose)
        body.SetTransform(Tz)
        


In [5]:
def add_machine4(model_name, x, y, rot_angle):
    '''
    Positions the machine type 4 in the environment
    model_name: KinBody name of the model
    x: x coordinate to place the machine's origin at
    y: y coordinate to place the machine's origin at
    rot_angle: rotation of the model
    '''
    global env

    body = env.GetKinBody(model_name)
    pose = openravepy.poseFromMatrix(body.GetTransform())
    pose[4] = x
    pose[5] = y

    Tz = openravepy.matrixFromPose(pose)
    body.SetTransform(Tz)
    
    body_aabb = body.ComputeAABB()
    body_x_len = body_aabb.extents()[0]*2
    body_y_len = body_aabb.extents()[1]*2
    body_z_len = body_aabb.extents()[2]*2

    # rot_angle = rot_angle - (np.pi / 2)

    # rotate around z axis
    rotation_matrix = np.identity(4)
    rotation_matrix[0][0] = math.cos(rot_angle)
    rotation_matrix[0][1] = -(math.sin(rot_angle))
    rotation_matrix[1][0] = math.sin(rot_angle)
    rotation_matrix[1][1] = math.cos(rot_angle)

    # after rotating need to shift it down since rotation origin is at top left corner
    if rot_angle == 0:
        pass
        # rotation_matrix[1][3] = -body_y_len
    elif rot_angle == np.pi / 2:
        # rotation_matrix[0][3] = -body_y_len
        rotation_matrix[1][3] = -body_x_len
    elif rot_angle == np.pi:
        rotation_matrix[0][3] = body_x_len
        rotation_matrix[1][3] = -body_y_len
    elif rot_angle == -np.pi / 2:
        rotation_matrix[0][3] = body_y_len
        # rotation_matrix[1][3] = -body_x_len

    body_trans = body.GetTransform()
    rotated_pose = np.matmul(body_trans, rotation_matrix)

    Tz = openravepy.matrixFromPose(rotated_pose)
    body.SetTransform(Tz)


In [6]:
# Room setup
# 4 rows 2 columns
rot_angles = [0, np.pi/2, np.pi, -np.pi/2]
machine_models = {
    'machine1': MODELS_DIR + 'machine1.dae',
    'machine2': MODELS_DIR + 'machine2.dae',
    'machine3': MODELS_DIR + 'machine3.dae',
    'machine4': MODELS_DIR + 'machine4.dae'
}

machine_base_model = MODELS_DIR + 'machine_base.dae'
workstation_model = MODELS_DIR + 'machine_base.dae'

machine_locations = {
    1: { 
        'room1': { 'x1': 9, 'x2': 3, 'y1': -3, 'y_inc': -5 },
        'room2': { 'x1': -10, 'x2': -4, 'y1': -3, 'y_inc': -5 },
    },
    2: {
        'room1': { 'x1': 9, 'x2': 3, 'y1': -3, 'y_inc': -5 },
        'room2': { 'x1': 9, 'x2': 3, 'y1': 20, 'y_inc': -5 },
    },
    3: {
        'room1': { 'x1': 9, 'x2': 3, 'y1': -3, 'y_inc': -5 },
        'room2': { 'x1': 17, 'x2': 23, 'y1': -3, 'y_inc': -5 },
    },
    4: {
        'room1': { 'x1': 9, 'x2': 3, 'y1': -3, 'y_inc': -5 },
        'room2': { 'x1': 9, 'x2': 3, 'y1': -25, 'y_inc': -5 },
    }
}

machine_range = {
    'room1' : [1, 8],
    'room2' : [9, 16]
}

environment_configuration = {'config' : config, 'machines' : {} }

for room_type in ['room1', 'room2']:
    x = machine_locations[config][room_type]['x1']
    y = machine_locations[config][room_type]['y1']

    for i in range(machine_range[room_type][0], machine_range[room_type][1] + 1):
        
        # Choose a random machine type to add
        # machine_type = machine_models.keys()[np.random.randint(0, len(machine_models))]
        if i == 1:
            machine_type = 'machine3'
        elif i == 2:
            machine_type = 'machine2'
        elif i == 3:
            machine_type = 'machine1'
        elif i == 4:
            machine_type = 'machine4'
        else:
            machine_models.pop('machine4', None)
            machine_type = machine_models.keys()[np.random.randint(0, len(machine_models))]

        idx = np.random.randint(0, len(rot_angles))
        rot_angle = rot_angles[idx]
        machine_base_name = None
        workstation_name = None

        # Generate machine model
        machine_name = 'machine{}'.format(i)
        dae15, dae14, sdf = MFG.generate_model_files(machine_name, machine_models[machine_type], CURRENT_ENV_DIR, MODELS_DIR)
        model_files[machine_name] = { 'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
        machine = env.ReadKinBodyURI(dae15)
        env.AddKinBody(machine)

        if machine_type == 'machine4':
            add_machine4(machine_name, x, y, rot_angle)

        else:            
            # Generate machine base
            machine_base_name = 'machine{}_base'.format(i)
            dae15, dae14, sdf = MFG.generate_model_files(machine_base_name, machine_base_model, CURRENT_ENV_DIR, MODELS_DIR)
            model_files[machine_base_name] = { 'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
            machine_base = env.ReadKinBodyURI(dae15)
            env.AddKinBody(machine_base)
                
            # Generate workstation (machinetype=machine3 does not require a workstation)
            if machine_type != 'machine3':
                workstation_name = 'workstation{}'.format(i)
                dae15, dae14, sdf = MFG.generate_model_files(workstation_name, workstation_model, CURRENT_ENV_DIR, MODELS_DIR)
                model_files[workstation_name] = { 'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
                workstation = env.ReadKinBodyURI(dae15)
                env.AddKinBody(workstation)

            add_machine_to_env(machine_base_name, x, y, rot_angle, workstation_name, machine_name)

        environment_configuration['machines'][machine_name] = { 
            "type" : machine_type, 
            "rot_angle" : rot_angle,
            "machine_base_name" : machine_base_name,
            "workstation_name" : workstation_name,
            "x" : x,
            "y" : y
        }

        print machine_type, machine_name, machine_base_name, workstation_name, x, y, rot_angle
        y += machine_locations[config][room_type]['y_inc']
        if i == machine_range[room_type][0] + 3:  # start next column
            x = machine_locations[config][room_type]['x2']
            y = machine_locations[config][room_type]['y1']

print config

machine3 machine1 machine1_base None 9 -3 0
machine2 machine2 machine2_base workstation2 9 -8 0
machine1 machine3 machine3_base workstation3 9 -13 -1.57079632679
machine4 machine4 None None 9 -18 1.57079632679
machine1 machine5 machine5_base workstation5 3 -3 1.57079632679
machine3 machine6 machine6_base None 3 -8 3.14159265359
machine1 machine7 machine7_base workstation7 3 -13 0
machine1 machine8 machine8_base workstation8 3 -18 3.14159265359
machine1 machine9 machine9_base workstation9 9 20 0
machine1 machine10 machine10_base workstation10 9 15 1.57079632679
machine2 machine11 machine11_base workstation11 9 10 1.57079632679
machine1 machine12 machine12_base workstation12 9 5 0
machine2 machine13 machine13_base workstation13 3 20 1.57079632679
machine1 machine14 machine14_base workstation14 3 15 1.57079632679
machine2 machine15 machine15_base workstation15 3 10 0
machine3 machine16 machine16_base None 3 5 1.57079632679
2


In [7]:
# Add ply object
ply_object_model = MODELS_DIR + 'ply_object.dae'
ply_object_model_name = 'ply_object'

dae15, dae14, sdf = MFG.generate_model_files(ply_object_model_name, ply_object_model, CURRENT_ENV_DIR, MODELS_DIR)
model_files[ply_object_model_name] = { 'dae15': dae15, 'dae14': dae14, 'sdf': sdf}
ply_object = env.ReadKinBodyURI(dae15)
env.AddKinBody(ply_object)

# Add ply object
model_name = 'tool_shelf'
shelf = env.GetKinBody(model_name)
shelf_aabb = shelf.ComputeAABB()
shelf_x = shelf_aabb.pos()[0]
shelf_y = shelf_aabb.pos()[1]
shelf_x_len = shelf_aabb.extents()[0] * 2
shelf_y_len = shelf_aabb.extents()[1] * 2
shelf_z_len = shelf_aabb.extents()[2] * 2

ply_object = env.GetKinBody(ply_object_model_name)
ply_aabb = ply_object.ComputeAABB()
ply_x = ply_aabb.pos()[0]
ply_y = ply_aabb.pos()[1]
ply_x_len = ply_aabb.extents()[0] * 2
ply_y_len = ply_aabb.extents()[1] * 2
ply_z_len = ply_aabb.extents()[2] * 2

x = shelf_x
y = shelf_y + ply_y_len/2
t = openravepy.matrixFromPose([1, 0, 0, 0, x, y, shelf_z_len+0.1])
ply_object.SetTransform(t)

# rotate around z axis
rot_angle = 0
# for config=4 (second room to the SOUTH) the tool shelf is placed to the north so we rotate the ply object
if config == 4:
    rot_angle = np.pi

object_rotation_matrix = np.identity(4)
object_rotation_matrix[0][0] = math.cos(rot_angle)
object_rotation_matrix[0][1] = -(math.sin(rot_angle))
object_rotation_matrix[1][0] = math.sin(rot_angle)
object_rotation_matrix[1][1] = math.cos(rot_angle)

if config == 4:
    object_rotation_matrix[0][3] = shelf_y_len/2
    object_rotation_matrix[1][3] = -ply_x_len

object_trans = ply_object.GetTransform()
rotated_pose = np.matmul(object_trans, object_rotation_matrix)
Tz = openravepy.matrixFromPose(rotated_pose)
ply_object.SetTransform(Tz)



# Translate all models to load robot at (0,0,0)

In [8]:
model_name = 'room1_floor'
floor = env.GetKinBody(model_name)
floor_aabb = floor.ComputeAABB()
floor_x_len = floor_aabb.extents()[0]
floor_y_len = floor_aabb.extents()[1]
floor_z_len = floor_aabb.extents()[2]
floor_x = floor_aabb.pos()[0]
floor_y = floor_aabb.pos()[1]

for body in env.GetBodies():

    body_transform = body.GetTransform()
    pose = openravepy.poseFromMatrix(body.GetTransform())
    pose[4] -= floor_x_len
    pose[5] += floor_y_len
    Tz = openravepy.matrixFromPose(pose)
    body.SetTransform(Tz)


In [9]:
# Add robot
# FetchOpenRaveRobotModel(env, False)

In [10]:
env.Save(ENV_GEN_DIR + 'env.dae')
json.dump(environment_configuration, open(ENV_GEN_DIR + 'env_config.json','w'))
json.dump(model_files, open(ENV_GEN_DIR + 'model_files.json', 'w'))

# Load Openrave Environment to Gazebo

In [1]:
# Run `rosrun gazebo_ros gazebo` before execution

import rospy, tf
from gazebo_msgs.srv import DeleteModel, SpawnModel, SetModelState
from geometry_msgs.msg import *
from gazebo_msgs.msg import ModelState 
import json
import openravepy
import Config
ENV_GEN_DIR = Config.TEST_DIR + 'EnvGenerators/' + Config.DOMAIN + '/'

env = openravepy.Environment()
env.SetViewer('qtcoin')
env.Load(ENV_GEN_DIR + 'env.dae')

model_files = json.load(open(ENV_GEN_DIR + 'model_files.json', 'r'))

def spawn(model_name, model_file, pose):
    with open(model_file, "r") as f:
        model = f.read()

    initial_pose = Pose()
    initial_pose.position.x = pose[4]
    initial_pose.position.y = pose[5]
    initial_pose.position.z = pose[6]
    initial_pose.orientation.x = pose[1]
    initial_pose.orientation.y = pose[2]
    initial_pose.orientation.z = pose[3]
    initial_pose.orientation.w = pose[0]
    
    state_msg = ModelState()
    state_msg.model_name = model_name
    state_msg.pose.position.x = pose[4]
    state_msg.pose.position.y = pose[5]
    state_msg.pose.position.z = pose[6]
    state_msg.pose.orientation.x = pose[1]
    state_msg.pose.orientation.y = pose[2]
    state_msg.pose.orientation.z = pose[3]
    state_msg.pose.orientation.w = pose[0]

    print model_name
    
    spawn_model(model_name, model, "", initial_pose, "world")
    set_state(state_msg)

# RUN `rosrun gazebo_ros gazebo`
print("Waiting for gazebo services...")
rospy.init_node("spawner")

# rospy.wait_for_service("gazebo/delete_model")
rospy.wait_for_service("gazebo/spawn_sdf_model")
rospy.wait_for_service('/gazebo/set_model_state')

set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
# delete_model = rospy.ServicePoxy("gazebo/delete_model", DeleteModel)
spawn_model = rospy.ServiceProxy("gazebo/spawn_sdf_model", SpawnModel)

for body_name in model_files.keys():
    body = env.GetKinBody(body_name)
    body_pose = openravepy.poseFromMatrix(body.GetTransform())
    spawn(body_name, model_files[body_name]['sdf'], body_pose)


Waiting for gazebo services...
machine11_base
tool_shelf
room2_floor
machine7_base
room2_walls
workstation3
machine10_base
machine14
workstation2
room1_floor
machine13_base
machine1_base
machine8_base
machine16_base
machine12_base
machine2_base
machine15_base
machine15
workstation6
workstation5
machine16
machine11
machine10
machine13
machine12
machine9_base
machine5_base
workstation9
ply_object
machine9
machine8
machine14_base
machine5
machine4
machine7
machine6
machine1
machine3
machine2
workstation13
workstation12
workstation10
workstation15
workstation14
machine6_base
machine3_base
room1_walls
